#Define persona


In [5]:
SYS_PROMPT_GEN = "You are a helpful AI Assistant"
SYS_PROMPT_1 = "You are a helpful medical assistant"
SYS_PROMPT_2 = "You are an experienced general physician"

domain_expert = 'dermatologist'
SYS_PROMPT = f"You are a knowlegeable {domain_expert} with 30 years of experience."

In [1]:
import pandas as pd
import os

COT_1 = "think step-by-step"
domain = "skin"

disease_list = ["Acne", "Fungal infection", "Impetigo", "Psoriasis", "Chicken pox", "others"]

current_dir = os.path.dirname(os.path.abspath('__file__'))
data_frame = pd.read_csv(os.path.join(current_dir, 'resources/Symptom2Disease.csv'))

filtered_df = data_frame[data_frame['label'].isin(disease_list)]
#rename column 0 to id 
filtered_df = filtered_df.rename(columns={filtered_df.columns[0]: 'id'})

#choose a random 20 rows to sample 
filtered_df = filtered_df.sample(n=20, random_state=1)

#Prompt examples

In [3]:
def get_query_prompt(symptom, disease_list, system_prompt):
    """
    This function generates a query prompt for the AI model.
    
    Parameters:
    symptom (str): The symptom to be analyzed.
    disease_list (list): The list of diseases to consider.

    Returns:
    list: A list containing the system and user prompts.
    """
    
    # Generate the message

    message = [
                {
                'role': "system", 
                'content': system_prompt
                },
                {
                "role": "user",
                "content": f"""Q: A patient comes into the clinic describing the symptom '{symptom}' , \
                        Determine the likelihood in percentage of severity of disease involved from the given list '{disease_list}'. \
                        ----- \
                        SCHEMA  The output (Likelihood) should be formatted as a JSON object. \
                        ----- \
                        'Likelihood': [Acne: a%, Fungal infection: b%, Impetigo: c%, Psoriasis: d%, Chicken pox: e%, others: f% ] \
                        Please answer in unbaised manner and in case when you do not have the information about any disease \
                        , please mention it as '0%' likelihood estimate. \
                        Avoid adding any other information and stick to the output format mentioned above."""          
              }      
    ]
    return message


In [ ]:
import sys
import os

# Add the parent directory (semiollm) to Python path
sys.path.append(os.path.dirname(os.path.abspath('')))
from src.models.models import OpenAIModel
from src.helper.result_parser import form_json

def get_model_response(message, llm_model):
    if 'gpt' in llm_model or 'davinci' in llm_model :
        sys_prompt = message[0]
        input_text = message[1:]
        gpt= OpenAIModel(llm_model, max_new_tokens=800, temperature=0.2,
                                top_p=0.1, system_prompt=sys_prompt)
        answer = gpt.predict(input_text)
    return answer
    
# Create a dictionary to store the results
query_prompts = []
ground_truth = []
result_dictionary = {}
for index, row in filtered_df.iterrows():
    symptom = row['text']
    ground_truth.append(row['label'])
    id = row['id']
    #print(f"for id: {id} Symptom: {symptom} - Disease: {ground_truth}")
    # Generate the query prompt
    query_prompt = get_query_prompt(symptom, disease_list, SYS_PROMPT)
    query_prompts.append(query_prompt)
    answer = get_model_response(query_prompt, llm_model="gpt-4-1106-preview")
    res_dict = form_json(answer)
    result_dictionary[id] = res_dict
    

In [15]:
#the result_dictionary is the output generated
#for each key in the dictionary, get the value of 'Likelihood' and compare it with the ground truth

def eval_model(result_dictionary, ground_truth):
    """
    This function evaluates the model's predictions against the ground truth.
    
    Parameters:
    result_dictionary (dict): The dictionary containing model predictions.
    ground_truth (list): The list of actual diseases.

    Returns:
    None
    """
    pred = []

    #loop over result_dictionary and get the value of 'Likelihood' for each key
    for key, value in result_dictionary.items():
        try:
            #if result_dictionary is {} or None, say output is None
            if not value:
                print(f"for id: {key} - Likelihood: {value} and res: None")
                pred.append(None)
                continue
            output = -1
            likelihood = result_dictionary[key]['predicted_likelihood']
            for disorder, val in likelihood.items():
                try:
                    if '-' in val:
                        low, high = map(float, val.rstrip('%').split('-'))
                        res = (low + high) / 2
                    elif '%' in val:
                        res = float(val.rstrip('%'))
                except:
                    try:
                        res = float(val) # try converting without stripping '%'
                    except:
                        print(f"Error in converting string to float. Check the value {val} for key {key} and index {index}")
                        res = 0.0
                if res > output:    
                    output = res
                    outcome_disease = disorder  
        except:
            outcome_disease = None      
        print(f"for id: {key} - Likelihood: {likelihood} and result: {outcome_disease}")
        pred.append(outcome_disease)

    #compare pred and ground truth and print the accuracy
    correct = 0
    for i in range(len(pred)):
        if pred[i] == ground_truth[i]:
            correct += 1
    accuracy = correct / len(pred) * 100
    print(f"Accuracy: {accuracy}%")



        

In [16]:
eval_model(result_dictionary, ground_truth)

for id: 167 - Likelihood: {'Acne': '0%', 'Fungal infection': '0%', 'Impetigo': '10%', 'Psoriasis': '0%', 'Chicken pox': '70%', 'others': '20%'} and result: Chicken pox
for id: 299 - Likelihood: {'Acne': '30%', 'Fungal infection': '20%', 'Impetigo': '25%', 'Psoriasis': '5%', 'Chicken pox': '10%', 'others': '10%'} and result: Acne
for id: 280 - Likelihood: {'Acne': '40%', 'Fungal infection': '20%', 'Impetigo': '15%', 'Psoriasis': '5%', 'Chicken pox': '5%', 'others': '15%'} and result: Acne
for id: 11 - Likelihood: {'Acne': '0%', 'Fungal infection': '30%', 'Impetigo': '10%', 'Psoriasis': '20%', 'Chicken pox': '15%', 'others': '25%'} and result: Fungal infection
for id: 191 - Likelihood: {'Acne': '0%', 'Fungal infection': '0%', 'Impetigo': '0%', 'Psoriasis': '0%', 'Chicken pox': '10%', 'others': '90%'} and result: others
for id: 274 - Likelihood: {'Acne': '70%', 'Fungal infection': '10%', 'Impetigo': '5%', 'Psoriasis': '0%', 'Chicken pox': '0%', 'others': '15%'} and result: Acne
for id: 15

#TEST with different CoTs and personas and languages for in depth analysis

our results for random 20 samples

with expertise 

AI Assistant: 30.00% accuracy

Medical Assitant : 35.00% accuracy

general physician: 45.00% accuracy

dermatologist : 55.00% accuracy